## Import Libraries

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from sklearn.preprocessing import MinMaxScaler
from torchmetrics.functional.classification import *
import tensorflow as tf
from tensorflow import keras
from keras.utils import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from skmultilearn.ext import Keras

## Read Data

In [25]:
data = pd.read_csv('D:\Project\project-ednn\Code\Final_dataset_Diabetes_Complication.csv')
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X24,X25,X26,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,3,1,1,1,1,1,1,1,3,1,...,1,3,1,2,2,1,2,2,1,2
1,5,3,1,1,2,4,2,1,5,1,...,1,2,2,2,2,2,2,2,1,1
2,4,1,1,1,2,1,2,2,2,2,...,2,3,1,2,2,2,2,2,2,1
3,4,1,1,1,2,4,2,1,1,1,...,2,3,2,2,2,2,1,2,1,2
4,2,1,1,3,1,3,1,1,1,1,...,1,3,1,2,2,2,2,2,2,2


In [26]:
data = data.drop_duplicates(ignore_index=True)
data

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X24,X25,X26,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,3,1,1,1,1,1,1,1,3,1,...,1,3,1,2,2,1,2,2,1,2
1,5,3,1,1,2,4,2,1,5,1,...,1,2,2,2,2,2,2,2,1,1
2,4,1,1,1,2,1,2,2,2,2,...,2,3,1,2,2,2,2,2,2,1
3,4,1,1,1,2,4,2,1,1,1,...,2,3,2,2,2,2,1,2,1,2
4,2,1,1,3,1,3,1,1,1,1,...,1,3,1,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543770,1,1,2,1,1,4,1,1,5,2,...,2,3,1,2,2,2,2,2,2,2
543771,2,1,1,1,1,4,1,1,5,1,...,1,2,2,2,2,1,2,2,2,1
543772,1,2,1,1,1,3,1,1,2,1,...,1,3,1,2,2,2,2,2,2,1
543773,3,1,2,1,1,3,1,1,2,2,...,1,3,2,2,2,2,2,1,2,2


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 543775 entries, 0 to 543774
Data columns (total 33 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   X1      543775 non-null  int64
 1   X2      543775 non-null  int64
 2   X3      543775 non-null  int64
 3   X4      543775 non-null  int64
 4   X5      543775 non-null  int64
 5   X6      543775 non-null  int64
 6   X7      543775 non-null  int64
 7   X8      543775 non-null  int64
 8   X9      543775 non-null  int64
 9   X10     543775 non-null  int64
 10  X11     543775 non-null  int64
 11  X12     543775 non-null  int64
 12  X13     543775 non-null  int64
 13  X14     543775 non-null  int64
 14  X15     543775 non-null  int64
 15  X16     543775 non-null  int64
 16  X17     543775 non-null  int64
 17  X18     543775 non-null  int64
 18  X19     543775 non-null  int64
 19  X20     543775 non-null  int64
 20  X21     543775 non-null  int64
 21  X22     543775 non-null  int64
 22  X23     543775 non-n

## Data Visualization

In [28]:
data_feature = data.iloc[:, :26]
data_label = data.iloc[:, 26:]
data_label

,Y1,Y2,Y3,Y4,Y5,Y6,Y7
0,2,2,1,2,2,1,2
1,2,2,2,2,2,1,1
2,2,2,2,2,2,2,1
3,2,2,2,1,2,1,2
4,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
543770,2,2,2,2,2,2,2
543771,2,2,1,2,2,2,1
543772,2,2,2,2,2,2,1
543773,2,2,2,2,1,2,2


## Data Preprocessing

In [29]:
scaler = MinMaxScaler()
scaler.fit(data)

# Transform the DataFrame to obtain the normalized data
data_normalized = scaler.transform(data)

In [30]:
X = data_normalized[:, :26]
y = data_normalized[:, 26:]

In [31]:
data_normalize = pd.DataFrame(scaler.transform(data), columns=data.columns)
df_feature = data_normalize.iloc[:, :26]
df_label = data_normalize.iloc[:, 26:]

## Modelling

In [32]:
weight_list = []
for i, label in enumerate(df_label.columns):
    values_1 = df_label[label][df_label[label] == 1]
    values_2 = df_label[label][df_label[label] == 2]
    dist = 1 - (len(values_1) / len(df_label))
    weight_list.append(dist)

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [34]:
weights_ = torch.Tensor(weight_list).to(device)
weights_

tensor([0.0483, 0.0652, 0.0605, 0.0508, 0.1008, 0.3895, 0.4290],
       device='cuda:0')

In [35]:
def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            # print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [36]:
class DNNModelwithWeightedDropout(tf.keras.Model):
    def __init__(self, num_features=26, num_classes=7, p=[0.05, 0.01, 0.003]):
        super(DNNModelwithWeightedDropout, self).__init__()
        self.fcn1 = tf.keras.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape=(num_features,)),
            tf.keras.layers.Dense(len(p), activation='softmax')
        ])
        self.dropout = [tf.keras.layers.Dropout(rate=rate) for rate in p]
        self.fcn2 = tf.keras.Sequential([
            tf.keras.layers.Dense(num_classes, activation='sigmoid')
        ])

    def call(self, x, training=None, mask=None):
        x = self.fcn1(x)
        weights = x

        # Apply weighted dropout
        x = [dropout(x[:, i], training=training) * weights[:, i] for i, dropout in enumerate(self.dropout)]

        x = tf.stack(x, axis=1)
        x = self.fcn2(x)
        return x

In [37]:
class DNNDropout(tf.keras.Model):
    def __init__(self, num_features=26, num_classes=7, p=0.5):
        super(DNNDropout, self).__init__()

        self.fcn1 = tf.keras.Sequential([
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dense(8, activation='relu'),
            tf.keras.layers.Dense(3, activation='softmax')
        ])

        self.dropout1 = tf.keras.layers.Dropout(p)

        self.fcn2 = tf.keras.Sequential([
            tf.keras.layers.Dense(num_classes, activation='sigmoid')
        ])

    def call(self, x, training=False):
        x = self.fcn1(x)
        x = self.dropout1(x)
        x = self.fcn2(x)
        return x

In [38]:
def DropoutModel(dropout_rate):
    model = DNNDropout(num_features=26, num_classes=7, p=dropout_rate)
    lossWeights = weight_list
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), metrics=['accuracy'], loss_weights=lossWeights)

    return model

In [39]:
def ProposedModel(dropout_rate):
    model = DNNModelwithWeightedDropout(num_features=26, num_classes=7, p=dropout_rate)
    lossWeights = weight_list
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), metrics=['accuracy'], loss_weights=lossWeights)

    return model

## Metrics

In [40]:
import numpy as np
import scipy.io as sci

def findmax(outputs):
    if len(outputs) == 0:
        return None, None  # Handle the case when outputs is empty

    Max = -float("inf")
    index = 0

    for i in range(len(outputs)):
        if outputs[i].float() > Max:
            Max = outputs[i]
            index = i

    return Max, index

def OneError(outputs, test_target):
    test_data_num = outputs.shape[0]
    class_num = outputs.shape[1]
    num = 0
    one_error = 0
    for i in range(test_data_num):
        if sum(test_target[i]) != class_num and sum(test_target[i]) != 0:
            Max, index = findmax(outputs[i])
            num = num + 1
            if test_target[i][index] != 1:
                one_error = one_error + 1
    return one_error / num

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        x = sum(np.logical_and(y_true[i], y_pred[i]))
        y = sum(np.logical_or(y_true[i], y_pred[i]))
        temp +=  x/y if not np.isnan(x/y) else 0
    return temp / y_true.shape[0]

def Recall(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_true[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_true[i])
    return temp/ y_true.shape[0]

def Precision(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if sum(y_pred[i]) == 0:
            continue
        temp+= sum(np.logical_and(y_true[i], y_pred[i]))/ sum(y_pred[i])
    return temp/ y_true.shape[0]

In [41]:
def mlc_metrics(trues, preds, test_name):
    subset_acc = multilabel_exact_match(preds, trues, num_labels=7).item()
    hamming_loss = multilabel_hamming_distance(preds, trues, num_labels=7).item()
    acc_exp = Accuracy(preds, trues)
    prec_exp = Precision(preds, trues)
    rec_exp = Recall(preds, trues)
    f1_exp = multilabel_fbeta_score(preds, trues, beta=1.0, num_labels=7).item()
    one_error = OneError(preds, trues)
    cov = multilabel_coverage_error(preds.float(), trues, num_labels=7).item()
    rank_loss = multilabel_ranking_loss(preds.float(), trues, num_labels=7).item()
    avg_prec = multilabel_average_precision(preds.float(), trues.long(), num_labels=7).item()
    acc_micro = multilabel_accuracy(preds, trues, num_labels=7, average='micro').item()
    acc_macro = multilabel_accuracy(preds, trues, num_labels=7, average='macro').item()
    prec_micro = multilabel_precision(preds, trues, num_labels=7, average='micro').item()
    prec_macro = multilabel_precision(preds, trues, num_labels=7, average='macro').item()
    rec_micro = multilabel_recall(preds, trues, num_labels=7, average='micro').item()
    rec_macro = multilabel_recall(preds, trues, num_labels=7, average='macro').item()
    f1_micro = multilabel_f1_score(preds, trues, num_labels=7, average='micro').item()
    f1_macro = multilabel_f1_score(preds, trues, num_labels=7, average='macro').item()
    auc_micro = multilabel_auroc(preds.float(), trues.long(), num_labels=7, average='micro').item()
    auc_macro = multilabel_auroc(preds.float(), trues.long(), num_labels=7, average='macro').item()

    data = {
        "Test Params": test_name,
        "Subset Accuracy": subset_acc,
        "Hamming Loss": hamming_loss,
        "Accuracy (Exp)": acc_exp,
        "Precision (Exp)": prec_exp,
        "Recall (Exp)": rec_exp,
        "F1 Score (Exp)": f1_exp,
        "One Error": one_error,
        "Coverage": cov,
        "Ranking Loss": rank_loss,
        "Average Precision": avg_prec,
        "Accuracy (Micro)": acc_micro,
        "Accuracy (Macro)": acc_macro,
        "Precision (Micro)": prec_micro,
        "Precision (Macro)": prec_macro,
        "Recall (Micro)": rec_micro,
        "Recall (Macro)": rec_macro,
        "F1 Score (Micro)": f1_micro,
        "F1 Score (Macro)": f1_macro,
        "AUC (Micro)": auc_micro,
        "AUC (Macro)": auc_macro
    }

    df = pd.DataFrame([data])  # Creating a DataFrame with a single row
    return df

## Hyperparameter Tuning

In [42]:
from sklearn.model_selection import train_test_split
x_train, x_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
x_test, x_valid, y_test, y_valid = train_test_split(x_temp, y_temp, test_size=0.3, random_state=42)

In [43]:
def objective1(trial):
    keras.backend.clear_session()
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)

    model = DropoutModel(dropout_rate)

    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=10,
        callbacks=[TFKerasPruningCallback(trial, "val_loss")],
        validation_data=(x_valid, y_valid),
        verbose=1,
    )

    # Evaluate the model accuracy on the validation set.
    prediction = model.predict(x_test)
    trues = torch.tensor(y_test)
    preds = torch.tensor(prediction)

    df = mlc_metrics(trues, preds, test_name=dropout_rate)
    df.to_csv(f'Metrics/{dropout_rate}.csv', index=False)

    score = multilabel_accuracy(preds, trues, num_labels=7, average='micro').item()
    return score

In [44]:
def objective2(trial):
    keras.backend.clear_session()
    
    dropout_rate1 = trial.suggest_float('dropout_rate1', 0.0, 0.5)
    dropout_rate2 = trial.suggest_float('dropout_rate2', 0.0, 0.5)
    dropout_rate3 = trial.suggest_float('dropout_rate3', 0.0, 0.5)
    dropout_rate = [dropout_rate1, dropout_rate2, dropout_rate3]

    model = ProposedModel(dropout_rate)

    model.fit(
        x_train,
        y_train,
        batch_size=32,
        epochs=10,
        callbacks=[TFKerasPruningCallback(trial, "val_loss")],
        validation_data=(x_valid, y_valid),
        verbose=1,
    )

    # Evaluate the model accuracy on the validation set.
    prediction = model.predict(x_test)
    trues = torch.tensor(y_test)
    preds = torch.tensor(prediction)

    df = mlc_metrics(trues, preds, test_name=dropout_rate)
    df.to_csv(f'Metrics/{dropout_rate}.csv', index=False)

    score = multilabel_accuracy(preds, trues, num_labels=7, average='micro').item()
    return score

In [45]:
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(objective1, n_trials=50)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

[I 2024-01-25 13:10:11,658] A new study created in memory with name: no-name-0ed70c3b-5f32-4ca9-a364-8ce696bd41cf


Epoch 1/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0160 - accuracy: 0.3417 - val_loss: 0.0151 - val_accuracy: 0.3731
Epoch 2/10
11896/11896 [==============================] - 42s 4ms/step - loss: 0.0156 - accuracy: 0.3378 - val_loss: 0.0151 - val_accuracy: 0.3389
Epoch 3/10
11896/11896 [==============================] - 42s 4ms/step - loss: 0.0156 - accuracy: 0.3237 - val_loss: 0.0151 - val_accuracy: 0.3942
Epoch 4/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0156 - accuracy: 0.3412 - val_loss: 0.0151 - val_accuracy: 0.2516
Epoch 5/10
11896/11896 [==============================] - 42s 4ms/step - loss: 0.0156 - accuracy: 0.3204 - val_loss: 0.0151 - val_accuracy: 0.4611
Epoch 6/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0156 - accuracy: 0.3284 - val_loss: 0.0151 - val_accuracy: 0.4261
Epoch 7/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0156 - accuracy: 0.3282 - val_loss: 0

[I 2024-01-25 13:17:57,082] Trial 0 finished with value: 0.8679190874099731 and parameters: {'dropout_rate': 0.1771900798042112}. Best is trial 0 with value: 0.8679190874099731.


Epoch 1/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0157 - accuracy: 0.3355 - val_loss: 0.0150 - val_accuracy: 0.2315
Epoch 2/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0150 - accuracy: 0.2465 - val_loss: 0.0148 - val_accuracy: 0.1579
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0150 - accuracy: 0.2485 - val_loss: 0.0148 - val_accuracy: 0.1726
Epoch 4/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0150 - accuracy: 0.2494 - val_loss: 0.0148 - val_accuracy: 0.2256
Epoch 5/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0150 - accuracy: 0.2497 - val_loss: 0.0148 - val_accuracy: 0.1600
Epoch 6/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0150 - accuracy: 0.2495 - val_loss: 0.0148 - val_accuracy: 0.1953
Epoch 7/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0150 - accuracy: 0.2449 - val_loss: 0

[I 2024-01-25 13:25:44,695] Trial 1 finished with value: 0.8712468147277832 and parameters: {'dropout_rate': 0.035528495568592444}. Best is trial 1 with value: 0.8712468147277832.


...

[I 2024-01-25 16:28:04,769] Trial 47 pruned. Trial was pruned at epoch 1.


Epoch 1/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0160 - accuracy: 0.3482 - val_loss: 0.0152 - val_accuracy: 0.2114
Epoch 2/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0157 - accuracy: 0.3529 - val_loss: 0.0152 - val_accuracy: 0.2025
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0157 - accuracy: 0.3536 - val_loss: 0.0153 - val_accuracy: 0.4819
Epoch 4/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0158 - accuracy: 0.3537 - val_loss: 0.0152 - val_accuracy: 0.3160
Epoch 5/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0158 - accuracy: 0.3501 - val_loss: 0.0152 - val_accuracy: 0.3397
Epoch 6/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0158 - accuracy: 0.3615 - val_loss: 0.0152 - val_accuracy: 0.2647
Epoch 7/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0158 - accuracy: 0.3571 - val_loss: 0

[I 2024-01-25 16:35:52,390] Trial 48 finished with value: 0.8651618361473083 and parameters: {'dropout_rate': 0.22991619589038942}. Best is trial 1 with value: 0.8712468147277832.


Epoch 1/10
11896/11896 [==============================] - 45s 4ms/step - loss: 0.0159 - accuracy: 0.3596 - val_loss: 0.0150 - val_accuracy: 0.3330
Epoch 2/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0156 - accuracy: 0.3505 - val_loss: 0.0150 - val_accuracy: 0.2250
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0156 - accuracy: 0.3632 - val_loss: 0.0149 - val_accuracy: 0.1752
Epoch 4/10
11885/11896 [============================>.] - ETA: 0s - loss: 0.0156 - accuracy: 0.3473

[I 2024-01-25 16:38:46,565] Trial 49 pruned. Trial was pruned at epoch 3.


In [46]:
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

trial = study.best_trial
print("Best trial:")
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Save results to csv file
df = study.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
df = df.drop('state', axis=1)                 # Exclude state column
df = df.sort_values('value')                  # Sort based on accuracy
df.to_csv('optuna_results.csv', index=False)  # Save to csv file

# Display results in a dataframe
print("\nOverall Results (ordered by accuracy):\n {}".format(df))

# Find the most important hyperparameters
most_important_parameters = optuna.importance.get_param_importances(study, target=None)

# Display the most important hyperparameters
print('\nMost important hyperparameters:')
for key, value in most_important_parameters.items():
    print('  {}:{}{:.2f}%'.format(key, (15-len(key))*' ', value*100))


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  39
  Number of complete trials:  11
Best trial:
  Value:  0.8712468147277832
  Params: 
    dropout_rate: 0.035528495568592444

Overall Results (ordered by accuracy):
     number     value  params_dropout_rate  system_attrs_completed_rung_0  \
7        7  0.836547             0.326886                       0.017471   
3        3  0.853861             0.380199                       0.015589   
36      36  0.860282             0.251872                       0.015378   
2        2  0.861677             0.344770                       0.015423   
48      48  0.865162             0.229916                       0.015204   
32      32  0.866438             0.248044                       0.015210   
6        6  0.866757             0.136193                       0.015013   
9        9  0.866814             0.223732                       0.015153   
0        0  0.867919             0.177190                          

In [47]:
study2 = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study2.optimize(objective2, n_trials=50)
pruned_trials2 = study2.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials2 = study2.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

[I 2024-01-25 16:38:46,709] A new study created in memory with name: no-name-0d579fe2-42d1-4284-bac6-abeb6c1c5006


Epoch 1/10
11896/11896 [==============================] - 45s 4ms/step - loss: 0.0165 - accuracy: 0.4589 - val_loss: 0.0155 - val_accuracy: 0.0039
Epoch 2/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0160 - accuracy: 0.4212 - val_loss: 0.0154 - val_accuracy: 0.3391
Epoch 3/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0161 - accuracy: 0.4404 - val_loss: 0.0154 - val_accuracy: 0.0874
Epoch 4/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0161 - accuracy: 0.4385 - val_loss: 0.0155 - val_accuracy: 0.3026
Epoch 5/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0161 - accuracy: 0.4262 - val_loss: 0.0154 - val_accuracy: 0.1742
Epoch 6/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0161 - accuracy: 0.4296 - val_loss: 0.0154 - val_accuracy: 0.3219
Epoch 7/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0161 - accuracy: 0.4263 - val_loss: 0

[I 2024-01-25 16:46:41,370] Trial 0 finished with value: 0.8617591261863708 and parameters: {'dropout_rate1': 0.4195099878736405, 'dropout_rate2': 0.4530816554777814, 'dropout_rate3': 0.35115998102347445}. Best is trial 0 with value: 0.8617591261863708.


Epoch 1/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0161 - accuracy: 0.4340 - val_loss: 0.0151 - val_accuracy: 0.2493
Epoch 2/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3413 - val_loss: 0.0151 - val_accuracy: 0.3338
Epoch 3/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0155 - accuracy: 0.3539 - val_loss: 0.0150 - val_accuracy: 0.3580
Epoch 4/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3304 - val_loss: 0.0150 - val_accuracy: 0.2249
Epoch 5/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3143 - val_loss: 0.0151 - val_accuracy: 0.4409
Epoch 6/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3085 - val_loss: 0.0150 - val_accuracy: 0.1287
Epoch 7/10
11896/11896 [==============================] - 43s 4ms/step - loss: 0.0154 - accuracy: 0.3116 - val_loss: 0

[I 2024-01-25 16:54:31,684] Trial 1 finished with value: 0.8665930032730103 and parameters: {'dropout_rate1': 0.357353702091831, 'dropout_rate2': 0.05149992294239014, 'dropout_rate3': 0.47360242290897925}. Best is trial 1 with value: 0.8665930032730103.


...

[I 2024-01-25 20:14:28,639] Trial 47 pruned. Trial was pruned at epoch 1.


Epoch 1/10
11896/11896 [==============================] - 45s 4ms/step - loss: 0.0161 - accuracy: 0.4484 - val_loss: 0.0151 - val_accuracy: 0.4072
Epoch 2/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0157 - accuracy: 0.4254 - val_loss: 0.0152 - val_accuracy: 0.5276
Epoch 3/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0156 - accuracy: 0.3689 - val_loss: 0.0151 - val_accuracy: 0.4945
Epoch 4/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0156 - accuracy: 0.3656 - val_loss: 0.0151 - val_accuracy: 0.3489
Epoch 5/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0156 - accuracy: 0.3528 - val_loss: 0.0150 - val_accuracy: 0.3393
Epoch 6/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0156 - accuracy: 0.3598 - val_loss: 0.0150 - val_accuracy: 0.3274
Epoch 7/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0156 - accuracy: 0.3586 - val_loss: 0

[I 2024-01-25 20:21:52,448] Trial 48 pruned. Trial was pruned at epoch 9.


Epoch 1/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0160 - accuracy: 0.3528 - val_loss: 0.0151 - val_accuracy: 0.5202
Epoch 2/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0154 - accuracy: 0.3034 - val_loss: 0.0151 - val_accuracy: 0.4474
Epoch 3/10
11896/11896 [==============================] - 44s 4ms/step - loss: 0.0154 - accuracy: 0.3182 - val_loss: 0.0150 - val_accuracy: 0.3329
Epoch 4/10
11885/11896 [============================>.] - ETA: 0s - loss: 0.0154 - accuracy: 0.3287

[I 2024-01-25 20:24:48,397] Trial 49 pruned. Trial was pruned at epoch 3.


In [48]:
print("\nStudy statistics: ")
print("  Number of finished trials: ", len(study2.trials))
print("  Number of pruned trials: ", len(pruned_trials2))
print("  Number of complete trials: ", len(complete_trials2))

trial = study2.best_trial
print("Best trial:")
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# Save results to csv file
df = study2.trials_dataframe().drop(['datetime_start', 'datetime_complete', 'duration'], axis=1)  # Exclude columns
df = df.loc[df['state'] == 'COMPLETE']        # Keep only results that did not prune
df = df.drop('state', axis=1)                 # Exclude state column
df = df.sort_values('value')                  # Sort based on accuracy
df.to_csv('optuna_results.csv', index=False)  # Save to csv file

# Display results in a dataframe
print("\nOverall Results (ordered by accuracy):\n {}".format(df))

# Find the most important hyperparameters
most_important_parameters = optuna.importance.get_param_importances(study2, target=None)

# Display the most important hyperparameters
print('\nMost important hyperparameters:')
for key, value in most_important_parameters.items():
    print('  {}:{}{:.2f}%'.format(key, (15-len(key))*' ', value*100))


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  36
  Number of complete trials:  14
Best trial:
  Value:  0.8701596856117249
  Params: 
    dropout_rate1: 0.29748295149940013
    dropout_rate2: 0.23118793026803292
    dropout_rate3: 0.28858336870989054

Overall Results (ordered by accuracy):
     number     value  params_dropout_rate1  params_dropout_rate2  \
3        3  0.861240              0.484010              0.135741   
9        9  0.861680              0.450873              0.495398   
0        0  0.861759              0.419510              0.453082   
17      17  0.863734              0.414991              0.191142   
29      29  0.863955              0.390192              0.191508   
37      37  0.866045              0.316958              0.200525   
23      23  0.866335              0.302531              0.225756   
1        1  0.866593              0.357354              0.051500   
6        6  0.867067              0.356677              0.261